In [20]:
ENV["GRB_LICENSE_FILE"] = "/Users/solvibjornsson/Gurobi/gurobi.lic"

"/Users/solvibjornsson/Gurobi/gurobi.lic"

In [21]:
using XLSX
using DataFrames

# Read data from Excel file
xf = XLSX.readxlsx("facilityData.xlsx")



function read_column(sheet, firstrow, col)
    res = []
    row = firstrow
    while !ismissing(sheet[row,col])
        push!(res,sheet[row,col])
        row+=1
    end 
    return res
end


function Customer_data(xf, sheet_name)
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    customer = read_column(sheet, firstrow, col)

    col = 2
    latitude = read_column(sheet, firstrow, col)

    col = 3
    longitude = read_column(sheet, firstrow, col)

    col = 4
    visits = read_column(sheet, firstrow, col)

    rawdata = DataFrame(customer = customer, latitude = latitude, 
            longitude = longitude, visits = visits)
    return rawdata
end

function facilities_data(xf, sheet_name)
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    facility = read_column(sheet, firstrow, col)

    col = 2
    latitude = read_column(sheet, firstrow, col)

    col = 3
    longitude = read_column(sheet, firstrow, col)

    col = 4
    closing_cost = read_column(sheet, firstrow, col)
    
    col = 5
    capacity = read_column(sheet, firstrow, col)

    col = 6
    type = read_column(sheet, firstrow, col)

    rawdata = DataFrame(facility = facility, latitude = latitude, 
            longitude = longitude, closing_cost = closing_cost, capacity = capacity, type = type)
    return rawdata
end


customers_data = Customer_data(xf, "Customers")

facility_data = facilities_data(xf, "Fac")


Row,facility,latitude,longitude,closing_cost,capacity,type
,Any,Any,Any,Any,Any,Any
1,Zurich West,47.3837,8.5265,20000,0,480
2,Geneva South,46.1793,6.3196,32500,0,288
3,Luzern North,47.0813,8.3017,15000,0,270
4,Bern North,46.9612,7.4358,0,75000,240
5,Lausanne,46.5152,6.6522,0,56250,300
6,Geneva North,46.3886,6.2428,0,68750,260
7,Altdorf,46.879,8.6483,0,62500,200
8,Montreux,46.4353,6.9167,0,37500,140
9,Zug,47.1525,8.5108,0,60000,160


# Routing Technicians within Z¨urich

In [22]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames

# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array, dropping the first row and column
    data = XLSX.getdata(sheet)#, drop=1:1, droprows=1)
    data=data[2:end,2:end]
    
    
    # Convert the array to a matrix
    #matrix_data = hcat(data...)
    return data
end

distances_matrix = read_excel_to_matrix("vrpData.xlsx", "distances")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
distances_matrix




31×31 Matrix{Any}:
  0         4         9.13017  10        …   2.72029   7.65506  1.23693
  4         0         8.00999   7.2111       6.40312   4.12311  2.08087
  9.13017   8.00999   0         4.11825     11.6499   10.9636   5.53624
 10         7.2111    4.11825   0           12.7201    8.63713  5.59732
  5.38516   2.23607  10.018     8.544        7.2111    2.28035  2.20907
  5         8.06226  14.0982   14.8661   …   2.52982  10.2956   1.96977
  5.09902   7.07107  14.0769   14.2127       3.60555   8.544    1.52643
  6.32456  10.198    14.4278   16.1245       3.82099  13.2136   3.30606
 12.7279   15.8114   13.4373   17.2627      12.5936   19.9249   7.45185
  4.24264   7.61577   8.93085  11.4018       4.66905  11.6362   3.245
  ⋮                                      ⋱                      ⋮
 12.0416   13        20.7162   20.1246      10.469    12.474    4.86621
 12.2703   11.6      19.6041   18.0488      11.7661    9.55196  4.90408
 18.4391   22.3607   25.4118   27.8568      15.9812  

In [30]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames



# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array
    data = XLSX.getdata(sheet)

    # Convert the array to a matrix and remove the first row and column
    # The `[2:end, 2:end]` slicing skips the first row and the first column
    matrix_data = convert(Matrix, data)[2:end, 2:end]

    return matrix_data
end

# Replace "vrpData.xlsx" with the actual path to your Excel file
distances_matrix = read_excel_to_matrix("vrpData.xlsx", "distances")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

function vrp_load_constraint()

    #------
    # DATA
    #------

    K = 10 # number vehicles
    N = 31; # number of nodes, node 1 is the depot

    C = 6; #capa for working hours excluding driving

    d = repair_times

    c = distances_matrix
    
    #------
    # MODEL
    #------

    model = Model(Gurobi.Optimizer);

    @variable(model, x[1:N,1:N,1:K] >= 0, Bin); #node to node coverd by truck
    @variable(model, y[1:N,1:K] >= 0, Bin); #
    @variable(model, z[1:N,1:K] >= 0); # load of truck k when arriving at node i

    @objective(model, Min, sum( sum( sum( c[i,j]*x[i,j,k]  for j in 1:N) for i in 1:N) for k = 1:K)); #mimize distance covered
    

    @constraint(model,[i = 2:N], sum(y[i,k] for k in 1:K) == 1); #every cust visited
    @constraint(model,[h = 1:N, k = 1:K], sum(i == h ? 0 : x[i,h,k] for i = 1:N) == y[h,k]); #inflow = outflow
    @constraint(model,[h = 1:N, k = 1:K], sum(j == h ? 0 : x[h,j,k] for j = 1:N) == y[h,k]); #inflow = outflow
    @constraint(model,[k = 1:K], sum(d[i]*y[i,k] for i = 2:N) <= C); #not exceeding work hours capa

    @constraint(model,[i = 2:N, j = 2:N, k = 1:K], z[i,k] - d[i] >= z[j,k] - (1-x[i,j,k])*sum(d));  #load constraint - subtour elimi

    #-------
    # SOLVE
    #-------
    set_optimizer_attribute(model, "TimeLimit", 30);
    optimize!(model)
    

    println();
    for k = 1:K
        has_route = false;
        for i = 1:N
            for j = 1:N
                if (value(x[i,j,k]) == 1)
                    print(i-1,"->",j-1," ")
                    has_route = true;
                end
            end
        end
        if has_route
            println();
        end
    end

    print("\ntotal cost = ");
    println(objective_value(model));

end

vrp_load_constraint();




Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.6.0 22G436)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9660 rows, 10230 columns and 46220 nonzeros
Model fingerprint: 0x7e32c2db
Variable types: 310 continuous, 9920 integer (9920 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [6e-01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 300 rows and 320 columns
Presolve time: 0.03s
Presolved: 9360 rows, 9910 columns, 45920 nonzeros
Variable types: 300 continuous, 9610 integer (9610 binary)

Root relaxation: objective 1.131945e+02, 990 iterations, 0.02 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

10. Location-based heuristic

In [29]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames



# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array
    data = XLSX.getdata(sheet)

    # Convert the array to a matrix and remove the first row and column
    # The `[2:end, 2:end]` slicing skips the first row and the first column
    matrix_data = convert(Matrix, data)[2:end, 2:end]

    return matrix_data
end


# Replace "vrpData.xlsx" with the actual path to your Excel file

#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
c_ij = read_excel_to_matrix("vrpData.xlsx", "c_ij_new")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

using JuMP
using Gurobi

function solve_facility_location()

    #------
    # DATA
    #------

    n = 30; 
    m = 30; # number of customers

    #f = Parameter, the cost of selectiong cust n as a cluster rep
    f = [8	18.26033954	20	10.77032961	10	10.19803903	12.64911064	25.45584412	8.485281374	17.11724277	21.28003759	18.02775638	4.472135955	24.75883681	12.16552506	13.41640786	21.9317122	25.61249695	39.59797975	34	22.91200559	24.08318916	24.54057864	36.87817783	22.36067977	27.91128804	9.666436779	5.440588203	15.31012737	2.473863375];
    # c: Parameter, the cost of assigning cust n to the cluster that cust m represents
    c = read_excel_to_matrix("vrpData.xlsx", "c_ij_new");
    d = read_excel_to_matrix("vrpData.xlsx", "repairTimes");
    d=d[2:end]; #exclude the depot

    C = 6; #we have γ = 1, so same capacity

    #------
    # MODEL
    #------

    model = Model(Gurobi.Optimizer);

    @variable(model, x[1:n] >= 0, Bin); #1 if cust n is chosen as "cluster representative"
    @variable(model, y[1:n,1:m] >= 0, Bin); #cust n is assigned to cluster that cust m represnts

    @objective(model, Min, sum( f[j]*x[j] for j in 1:n) + sum( sum( c[i,j]*y[i,j] for j in 1:n) for i = 1:m));

    @constraint(model,[i = 1:m], sum(y[i,j] for j in 1:n) == 1);
    @constraint(model,[j = 1:n], sum( d[i]*y[i,j] for i = 1:m) <= C*x[j]);

    #-------
    # SOLVE
    #-------
    set_optimizer_attribute(model, "TimeLimit", 30);
    optimize!(model)

    println();
    for j = 1:n
        allocated = false;
        for i = 1:n
            if (value(y[i,j]) >= 0.99999)
                print((i+1)," ");
                allocated = true;
            end
        end
        if allocated
            println();
        end
    end

end

solve_facility_location();



Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Set parameter TimeLimit to value 30
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.6.0 22G436)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 930 columns and 1830 nonzeros
Model fingerprint: 0x08234484
Variable types: 0 continuous, 930 integer (930 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+00]
  Objective range  [3e-04, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 796.9481815
Presolve time: 0.00s
Presolved: 60 rows, 930 columns, 1830 nonzeros
Variable types: 0 continuous, 930 integer (930 binary)
Found heuristic solution: objective 628.7489657

Root relaxation: objective 8.167736e+01, 76 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     

After running the new service to customers for some time, Comma Computing notices that
customers would prefer to have a time window directly when booking a service visit. However,
making this possible would also require changing the allocation of customers to routes, as these
time windows now have to be considered in the VRP. Comma Computing ask you for help in
implementing this and provide a set of time windows for the previously considered instance in the
times sheet of the vrpData.xlsx file. Moreover, you can assume that traveling one kilometer
costs 2.5 minutes. As you believe that solving an exact formulation will be too time-consuming to
provide good solutions for this vehicle routing problem with time windows, you decide to use the
Clarke-Wright savings heuristic as a starting point.

sd

In [2]:
using DataFrames, XLSX, Dates

# Load the workbook
wb = XLSX.readxlsx("vrpData.xlsx")

# Get the specified sheets
sheet_cw = wb["cw"]
sheet_times = wb["times"]
sheet_repairTimes = wb["repairTimes"]
sheet_distance = wb["dis_min_round"]

# Read the data into an array and skip headers by adjusting range
data_cw = XLSX.getdata(sheet_cw, "A2:C436")
data_times = XLSX.getdata(sheet_times, "A2:C31")
data_repairTimes = XLSX.getdata(sheet_repairTimes, "A2:C32")
data_distance = XLSX.getdata(sheet_distance, "B2:AF32")  # Adjust the range to exclude headers

# Convert to DataFrame and assign column names
savings = DataFrame(data_cw, [:i, :j, :s_ij])
times = DataFrame(data_times, [:Customer, :Start, :End])
repair_times = DataFrame(data_repairTimes, [:Customer, :Service_Time_h, :Service_Time_m])
distance_matrix = DataFrame(data_distance, Symbol.(0:30))  # Assuming there are 30 customers




# Function to convert time strings to Time type
function parse_time(str)
    h, m, s = split(str, ":")
    return Time(parse(Int, h), parse(Int, m), parse(Int, s))
end

# Apply time parsing if times are strings
if typeof(times[1, :Start]) == String
    times[!, :Start] = parse_time.(times[!, :Start])
    times[!, :End] = parse_time.(times[!, :End])
end

# Ensure the 's_ij' column is of type Float64
if savings[1, :s_ij] isa Number  # Check if the first element is a number
    savings[!, :s_ij] = float.(savings[!, :s_ij])  # Convert all to Float64 if not already
elseif savings[1, :s_ij] isa String
    savings[!, :s_ij] = parse.(Float64, savings[!, :s_ij])  # Parse strings to Float64
end



function check_time_feasibility(route, times, repair_times, travel_time_matrix, max_duration_minutes)
    total_time = 0
    current_time = Time(0, 0)  # start of the day for simplification
    #current_time = times.Start[times.Customer .== route[1]][1]  # Starting time of the first customer

    println("Checking route: $route")

    for customer_idx in 2:length(route)
        prev_customer = route[customer_idx - 1]
        customer = route[customer_idx]
        customer_start = times.Start[times.Customer .== customer][1]
        customer_end = times.End[times.Customer .== customer][1]

        # Print detailed timing information


        # Assuming travel_time_matrix contains travel times in minutes as Float64
        travel_time_minutes = travel_time_matrix[prev_customer+1, customer+1]
        travel_time = Minute(travel_time_minutes)  # Convert minutes to `Minute` object

        service_time_minutes = repair_times.Service_Time_m[repair_times.Customer .== customer][1]
        service_time = Minute(service_time_minutes)  # Convert minutes to `Minute` object

        # Calculate earliest possible start time for this customer
        possible_start = max(current_time + travel_time, customer_start)
        # Print detailed timing information
        println("Customer $customer: Start - $possible_start, End - $(possible_start + service_time), Travel Time - $travel_time, Service Time - $service_time")


        if possible_start > customer_end
            return false  # Cannot service this customer within their time window
        end

        # Update the current time after servicing this customer
        current_time = possible_start + service_time
        total_time += travel_time_minutes + service_time_minutes  # Sum up the total time in minutes

        
        end
        if total_time <= max_duration_minutes
            println("Total route time: $total_time minutes (within limit)")
        else
            println("Total route time: $total_time minutes (exceeds limit!)")
        end
    
        return total_time <= max_duration_minutes
    end


# Initialize routes with each customer as a separate route
routes = [[i] for i in 1:nrow(times)]
#sort!(savings, :s_ij, rev=true)  # Sort savings in descending order


# Implement the Clarke-Wright algorithm with time window constraints
for row in eachrow(savings)
    i, j = row.i, row.j
    route_i = findfirst(route -> i in route, routes)
    route_j = findfirst(route -> j in route, routes)

    if route_i != route_j
        merged_route = vcat(routes[route_i], routes[route_j])
        if check_time_feasibility(merged_route, times,repair_times, distance_matrix, 6 * 60)  # 6 hours converted to minutes
            routes[route_i] = merged_route
            deleteat!(routes, route_j)
        end
    end
end

println("Final route checks:")
for (index, route) in enumerate(routes)
    println("\nRoute $index: $route")
    check_time_feasibility(route, times, repair_times, distance_matrix, 6 * 60)  # 6 hours in minutes
end


Checking route: [19, 20]
Customer 20: Start - 10:15:00, End - 11:33:00, Travel Time - 15 minutes, Service Time - 78 minutes
Total route time: 93 minutes (within limit)
Checking route: [19, 20, 24]
Customer 20: Start - 10:15:00, End - 11:33:00, Travel Time - 15 minutes, Service Time - 78 minutes
Customer 24: Start - 12:10:00, End - 13:40:00, Travel Time - 37 minutes, Service Time - 90 minutes
Total route time: 220 minutes (within limit)
Checking route: [23, 26]
Customer 26: Start - 15:06:00, End - 16:00:00, Travel Time - 6 minutes, Service Time - 54 minutes
Total route time: 60 minutes (within limit)
Checking route: [18, 23, 26]
Customer 23: Start - 15:00:00, End - 15:48:00, Travel Time - 11 minutes, Service Time - 48 minutes
Customer 26: Start - 15:54:00, End - 16:48:00, Travel Time - 6 minutes, Service Time - 54 minutes
Total route time: 119 minutes (within limit)
Checking route: [19, 20, 24, 21]
Customer 20: Start - 10:15:00, End - 11:33:00, Travel Time - 15 minutes, Service Time - 7

# debug


In [4]:
using DataFrames, XLSX, Dates

# Load the workbook
wb = XLSX.readxlsx("vrpData.xlsx")

# Get the specified sheets
sheet_cw = wb["cw"]
sheet_times = wb["times"]
sheet_repairTimes = wb["repairTimes"]
sheet_distance = wb["dis_min_round"]

# Read the data into an array and skip headers by adjusting range
data_cw = XLSX.getdata(sheet_cw, "A2:C436")
data_times = XLSX.getdata(sheet_times, "A2:C31")
data_repairTimes = XLSX.getdata(sheet_repairTimes, "A2:C32")
data_distance = XLSX.getdata(sheet_distance, "B2:AF32")  # Adjust the range to exclude headers

# Convert to DataFrame and assign column names
savings = DataFrame(data_cw, [:i, :j, :s_ij])
times = DataFrame(data_times, [:Customer, :Start, :End])
repair_times = DataFrame(data_repairTimes, [:Customer, :Service_Time_h, :Service_Time_m])
distance_matrix = DataFrame(data_distance, Symbol.(0:30))  # Assuming there are 30 customers




# Function to convert time strings to Time type
function parse_time(str)
    h, m, s = split(str, ":")
    return Time(parse(Int, h), parse(Int, m), parse(Int, s))
end

# Apply time parsing if times are strings
if typeof(times[1, :Start]) == String
    times[!, :Start] = parse_time.(times[!, :Start])
    times[!, :End] = parse_time.(times[!, :End])
end

# Ensure the 's_ij' column is of type Float64
if savings[1, :s_ij] isa Number  # Check if the first element is a number
    savings[!, :s_ij] = float.(savings[!, :s_ij])  # Convert all to Float64 if not already
elseif savings[1, :s_ij] isa String
    savings[!, :s_ij] = parse.(Float64, savings[!, :s_ij])  # Parse strings to Float64
end



function check_time_feasibility(route, times, repair_times, travel_time_matrix, max_duration_minutes)
    # Start with the service time of the first customer
    first_customer_service_time = repair_times.Service_Time_m[repair_times.Customer .== route[1]][1]
    total_time = first_customer_service_time
    current_time = Time(0, 0) + Minute(first_customer_service_time)  # Start the current time at the end of the first service

    println("Checking route: $route")

    # Since you've already handled the first customer above, start the loop from the second customer
    for customer_idx in 2:length(route)
        prev_customer = route[customer_idx - 1]
        customer = route[customer_idx]
        customer_start = times.Start[times.Customer .== customer][1]
        customer_end = times.End[times.Customer .== customer][1]

        # Assume travel_time_matrix contains travel times in minutes as Float64
        travel_time_minutes = travel_time_matrix[prev_customer + 1, customer + 1]
        travel_time = Minute(travel_time_minutes)  # Convert minutes to `Minute` object

        service_time_minutes = repair_times.Service_Time_m[repair_times.Customer .== customer][1]
        service_time = Minute(service_time_minutes)  # Convert minutes to `Minute` object

        # Calculate earliest possible start time for this customer
        possible_start = max(current_time + travel_time, customer_start)
        
        # Print detailed timing information
        println("Customer $customer: Start - $possible_start, End - $(possible_start + service_time), Travel Time - $travel_time, Service Time - $service_time")

        if possible_start > customer_end
            println("Cannot service customer $customer within their time window.")
            return false
        end

        # Update the current time and total time after servicing this customer
        current_time = possible_start + service_time
        total_time += travel_time_minutes + service_time_minutes
    end

    if total_time <= max_duration_minutes
        println("Total route time: $total_time minutes (within limit)")
    else
        println("Total route time: $total_time minutes (exceeds limit!)")
    end

    return total_time <= max_duration_minutes
end



# Initialize routes with each customer as a separate route
routes = [[i] for i in 1:nrow(times)]
#sort!(savings, :s_ij, rev=true)  # Sort savings in descending order


# Implement the Clarke-Wright algorithm with time window constraints
for row in eachrow(savings)
    i, j = row.i, row.j
    route_i = findfirst(route -> i in route, routes)
    route_j = findfirst(route -> j in route, routes)

    if route_i != route_j
        merged_route = vcat(routes[route_i], routes[route_j])
        if check_time_feasibility(merged_route, times,repair_times, distance_matrix, 6 * 60)  # 6 hours converted to minutes
            routes[route_i] = merged_route
            deleteat!(routes, route_j)
        end
    end
end

println("Final route checks:")
for (index, route) in enumerate(routes)
    println("\nRoute $index: $route")
    check_time_feasibility(route, times, repair_times, distance_matrix, 6 * 60)  # 6 hours in minutes
end


Checking route: [19, 20]
Customer 20: Start - 09:00:00, End - 10:18:00, Travel Time - 15 minutes, Service Time - 78 minutes
Total route time: 165 minutes (within limit)
Checking route: [19, 20, 24]
Customer 20: Start - 09:00:00, End - 10:18:00, Travel Time - 15 minutes, Service Time - 78 minutes
Customer 24: Start - 10:55:00, End - 12:25:00, Travel Time - 37 minutes, Service Time - 90 minutes
Total route time: 292 minutes (within limit)
Checking route: [23, 26]
Customer 26: Start - 14:00:00, End - 14:54:00, Travel Time - 6 minutes, Service Time - 54 minutes
Total route time: 108 minutes (within limit)
Checking route: [18, 23, 26]
Customer 23: Start - 15:00:00, End - 15:48:00, Travel Time - 11 minutes, Service Time - 48 minutes
Customer 26: Start - 15:54:00, End - 16:48:00, Travel Time - 6 minutes, Service Time - 54 minutes
Total route time: 173 minutes (within limit)
Checking route: [19, 20, 24, 21]
Customer 20: Start - 09:00:00, End - 10:18:00, Travel Time - 15 minutes, Service Time -

Excessive output truncated after 524320 bytes.

Customer 19: Start - 17:39:00, End - 18:51:00, Travel Time - 55 minutes, Service Time - 72 minutes
Cannot service customer 19 within their time window.
Checking route: [1, 16, 4, 29, 5, 6, 15, 12, 2, 25, 30]
Customer 16: Start - 09:00:00, End - 09:30:00, Travel Time - 15 minutes, Service Time - 30 minutes
Customer 4: Start - 09:41:00, End - 10:23:00, Travel Time - 11 minutes, Service Time - 42 minutes
Customer 29: Start - 11:00:00, End - 12:24:00, Travel Time - 6 minutes, Service Time - 84 minutes
Customer 5: Start - 13:00:00, End - 13:48:00, Travel Time - 26 minutes, Service Time - 48 minutes
Customer 6: Start - 13:54:00, End - 14:18:00, Travel Time - 6 minutes, Service Time - 24 minutes
Customer 15: Start - 14:46:00, End - 15:10:00, Travel Time - 28 minutes, Service Time - 24 minutes
Cannot service customer 15 within their time window.
Checking route: [15, 12, 2, 25, 30, 22, 18, 23, 26]
Customer 12: Start - 09:00:00, End - 09:30:00, Travel Time - 11 minutes, Service Time - 30 minutes